In [288]:

import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import random
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import matplotlib.pyplot as plt


In [289]:


class train_MnistDataset(Dataset):

    def __init__(self):
        #data loading
        train_xy = np.loadtxt('C:/Users/Marcel/Documents/PyTorch/kaggle/digit_recognizer/train.csv', delimiter=',',dtype=np.float32, skiprows=1) 
        self.train_x = torch.from_numpy(train_xy[:,1:])
        self.train_y = torch.from_numpy(train_xy[:,0]).long()
        self.n_samples = train_xy.shape[0]
    def __getitem__(self,index):
        x = self.train_x[index].view(1, 28, 28)
        y = self.train_y[index]
        return x, y
        
    def __len__(self):
        #len(dataset)
        return self.n_samples
    
class test_MnistDataset(Dataset):

    def __init__(self):
        #data loading
        train_x = np.loadtxt('C:/Users/Marcel/Documents/PyTorch/kaggle/digit_recognizer/test.csv', delimiter=',',dtype=np.float32, skiprows=1) 
        self.train_x = torch.from_numpy(train_x)
        self.n_samples = train_x.shape[0]
    def __getitem__(self,index):
        x = self.train_x[index].view(1, 28, 28)
        return x
        
    def __len__(self):
        #len(dataset)
        return self.n_samples
    
dataset = train_MnistDataset()
dataset_test = test_MnistDataset()

In [290]:
import torchdata.datapipes.iter as pipes
from torch.utils.data import DataLoader, random_split

#Training performance
train_size, test_size = int(len(dataset) * 0.8), len(dataset) - (int(len(dataset) * 0.8))
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_train, batch_test =  100, 100
train_dataloader = DataLoader(train_dataset, batch_size=batch_train, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_test)


# Used for submition 
kaggle_train_dataloader = DataLoader(dataset, batch_size= batch_train, shuffle=True)
kaggle_test_dataloader = DataLoader(dataset_test, batch_size= len(dataset_test))

In [291]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1,64, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(64,64, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(128, 128,kernel_size=3,padding=1)
        self.conv5 = nn.Conv2d(128, 192,kernel_size=3,padding=1)
        self.conv6 = nn.Conv2d(192, 192 ,kernel_size=5,padding=2)



        self.fc1 = None
        self.fc2 = nn.Linear(256,10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.conv5(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.conv6(x)
        x = F.relu(x) 
        x = F.max_pool2d(x,2, padding=1)
        x = x.flatten(1)   
        if self.fc1 is None:
            self.fc1 = nn.Linear(x.shape[1], 256)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x






In [292]:
def weights_init(layer_in):
    if isinstance(layer_in, nn.Linear):
        nn.init.kaiming_uniform_(layer_in.weight)
        layer_in.bias.data.fill_(0.0)

In [293]:
model = Model()
model.apply(weights_init)
optimizer = optim.Adam(model.parameters(), lr=0.01)



In [294]:
def train(epoch, dataloader):
    model.train()
    for batch_idx, (data, target) in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # Store resultsn   n  
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(dataset), loss.item()))

In [300]:
# kaggle used as flag to remove statistics
def test(dataloader, predictions = None):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        if predictions == None:
            for data, target in dataloader:
                output = model(data)
                pred = output.data.max(1, keepdim=True)[1]
                test_loss += F.nll_loss(output, target, size_average=False).item()
                correct += pred.eq(target.data.view_as(pred)).sum()
        else:       
            for data in dataloader:
                output = model(data)
                pred = output.data.max(1, keepdim=True)[1]
                predictions.append(pred)
                
    test_loss /= len(test_dataloader.dataset)
    if predictions == None:
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataloader.dataset),
            100. * correct / len(dataloader.dataset)))


In [296]:
# def kaggle_test(predictions):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data in kaggle_dataloader:
#             output = model(data)
#             pred = output.data.max(1, keepdim=True)[1]
            
#             predictions.append(pred)
#     test_loss /= len(kaggle_dataloader.dataset)
#     print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         test_loss, correct, len(kaggle_dataloader.dataset),
#         100. * correct / len(kaggle_dataloader.dataset)))


In [297]:
test(test_dataloader)

C:\Users\Marcel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.4190, Accuracy: 974/8400 (12%)



In [302]:

n_epochs = 10
for epoch in range(1, n_epochs + 1):
  train(epoch, kaggle_train_dataloader)
  # testing on the same data to get more info in the model
  test(test_dataloader)


Train Epoch: 1 [0/42000]	Loss: 0.044448
Train Epoch: 1 [1000/42000]	Loss: 0.023661
Train Epoch: 1 [2000/42000]	Loss: 0.126045
Train Epoch: 1 [3000/42000]	Loss: 0.130805
Train Epoch: 1 [4000/42000]	Loss: 0.169722
Train Epoch: 1 [5000/42000]	Loss: 0.148961
Train Epoch: 1 [6000/42000]	Loss: 0.042026
Train Epoch: 1 [7000/42000]	Loss: 0.181557
Train Epoch: 1 [8000/42000]	Loss: 0.045725


In [ ]:
predictions = []
test(kaggle_test_dataloader,predictions)


import pandas as pd
predictions = np.array(predictions).reshape(-1)
indexes = range(1,len(predictions)+1,1)
df = pd.DataFrame({'ImageId': indexes, 'Label': predictions} )
df.to_csv('kaggle_sub_1.csv', index=False)